## Notebook settings

In [1]:
%load_ext autoreload
%autoreload 2

%autosave 10

#%load_ext lab_black

Autosaving every 10 seconds


In [2]:
import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join("..")))

## Imports

In [3]:
from lfp_analysis.data import *

In [1]:
import numpy as np
import pandas as pd
import h5py

from pathlib import Path

import matplotlib.pyplot as plt

%matplotlib widget
figsize=(8,3)

In [3]:
len(np.ones((2,2)).shape)

(2, 2)

# Import Data & Pre-process:

In [5]:
PAT_ID = Path('ET1')
DATASET_NAME = Path('Pegboard_on')

DATA_PATH = Path("./../data")
SMR_FILE = Path("./../../ET_Tremor_data") / PAT_ID / DATASET_NAME.with_suffix('.smr')

In [6]:
importer = SmrImporter(SMR_FILE)

In [7]:
importer.ch_names

['Fcz',
 'Cz',
 'F3',
 'F4',
 'C3',
 'C4',
 'Cp3',
 'Cp4',
 'L0',
 'L2',
 'L3',
 'R0',
 'R1',
 'R2',
 'R3',
 'L02',
 'R02',
 'R13',
 'FIDL',
 'FLBL',
 'FIDR',
 'FLBR',
 'Aclx',
 'Acly',
 'Aclz',
 'Acrx',
 'Acry',
 'Acrz',
 'FrcL',
 'FrcR']

In [8]:
importer.data.shape

(30, 1561039)

## Behavioral data:

In [9]:
bipolar = (
    LFPData.from_importer(importer, ['L02',
 'R02',
 'R13',])
    .highpass()
    .limit(std_factor=4.5,plot=True,figsize=(8,8))
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
emg = (EMGData
       .from_importer(importer,emg_names=['FLBR'])
       .to_label(smoothing=0.3,percentile=45,use_hilbert=True,figsize=figsize))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
acc = (
    AccData.from_importer(importer, ['Aclx',
 'Acly',
 'Aclz',
 'Acrx',
 'Acry',
 'Acrz'])
    .to_magnitude()
    .highpass()
    .to_label(smoothing=0.58,percentile=75,figsize=figsize)
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
plt.close('all')

In [13]:
emg.label.shape, bipolar.data.shape

((1561039,), (3, 1561039))

In [14]:
bipolar.plot(figsize=figsize)
plt.plot(acc.time(),emg.label*500)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Save to H5:

In [15]:
task = 'Pegboard'
stim_cond = 'on'
if task == 'Pegboard' or task=='Pouring':
    label_source = acc
else:
    label_source = emg

DATASET_NAME = Path(task+'_'+stim_cond)

In [16]:
save_to_h5(
    DATA_PATH / "preprocessed" / PAT_ID / DATASET_NAME.with_suffix('.h5'),
    bipolar.data,
    label_source.label.astype(np.bool_),
)

## Export to WAV and CSV for label revision:

In [17]:
from scipy.io.wavfile import write

def label_to_csv(path, label):
    
    label[0]= 0 if label[0]==1 else label[0]
    label[-1]= 0 if label[-1]==1 else label[-1]

    diffed = np.diff(label)
    
    (ons,), (offs,) = np.where(diffed==1), np.where(diffed==-1)

    data_df = pd.DataFrame(np.stack([ons,offs]).T / 2048.0)
    data_df['label'] = 'ON'
    
    assert ons.shape == offs.shape, "Unequal amounts of mvmt. onsets and offsets detected in label"
    
    data_df.to_csv(path)
    

def rescale_channels(data,dtype=np.int16):
    amp = np.iinfo(dtype).max
    
    return dtype(data / (np.max(data,axis=-1,keepdims=True) + 0.000001) * amp)


def save_to_wav(path,data):
    write(path,2048,rescale_channels(data).T)
    

In [18]:
lfp_wav_path = DATA_PATH / "preprocessed" / PAT_ID / Path(str(DATASET_NAME) + '_label') / "lfps.wav"
acc_wav_path = DATA_PATH / "preprocessed" / PAT_ID / Path(str(DATASET_NAME) + '_label') / "acc.wav"
emg_wav_path = DATA_PATH / "preprocessed" / PAT_ID / Path(str(DATASET_NAME) + '_label') / "emg.wav"

label_csv_path = DATA_PATH / "preprocessed" / PAT_ID / Path(str(DATASET_NAME) + '_label') / "regions.csv"

In [19]:
save_to_wav(lfp_wav_path,bipolar.data)
label_to_csv(label_csv_path,label_source.label)

In [20]:
save_to_wav(acc_wav_path,acc.data)

In [21]:
save_to_wav(emg_wav_path,emg.data)